## 下载数据集

In [1]:
from modelscope.msdatasets import MsDataset
import json
import random

random.seed(42)

ds = MsDataset.load('krisfu/delicate_medical_r1_data', subset_name='default', split='train')
data_list = list(ds)
random.shuffle(data_list)

split_idx = int(len(data_list) * 0.9)

train_data = data_list[:split_idx]
val_data = data_list[split_idx:]

with open('../dataset/delicate_medical_r1/train.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

with open('../dataset/delicate_medical_r1/val.jsonl', 'w', encoding='utf-8') as f:
    for item in val_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"The dataset has been split successfully.")
print(f"Train Set Size：{len(train_data)}")
print(f"Val Set Size：{len(val_data)}")

/home/fine/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The dataset has been split successfully.
Train Set Size：2166
Val Set Size：241


## 加载模型和分词器

In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM

/home/fine/uv/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/data/download-model/Qwen3-0.6B"

In [3]:
# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,use_cache=False)
tokenizer.padding_side = "left"

## 调整数据的格式用于训练

In [4]:
import os
datset_path='../dataset/delicate_medical_r1/'
train_dataset_path = os.path.join(datset_path,"train.jsonl")
test_dataset_path = os.path.join(datset_path,"val.jsonl")

In [5]:
from datasets import load_dataset
import torch

In [6]:
dataset=load_dataset('json',data_files={
    'train':train_dataset_path,
    'validation':test_dataset_path
})

In [7]:
PROMPT = "你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。"
MAX_LENGTH = 2048

In [8]:
def process_func(example):
    """
    将数据集进行预处理
    """ 
    instruction = f"<|im_start|>system\n{PROMPT}<|im_end|>\n<|im_start|>user\n{example['question']}<|im_end|>\n<|im_start|>assistant\n"
    response = f"<think>{example['think']}</think>\n{example['answer']}<|im_end|>"

    # 对完整对话进行编码
    full_input = instruction + response
    
    # tokenize
    tokenized = tokenizer(
        full_input,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )
    
    # 对仅指令部分进行编码以计算标签位置
    instruction_ids = tokenizer(
        instruction,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        return_tensors=None
    )['input_ids']
    
    # 创建labels，instruction部分设为-100，response部分保持原值
    labels = tokenized['input_ids'].copy()
    instruction_len = len(instruction_ids)
    for i in range(instruction_len):
        if i < len(labels):
            labels[i] = -100
    
    tokenized['labels'] = labels
    
    return tokenized

In [9]:
dataset=dataset.map(process_func)

Map: 100%|██████████| 241/241 [00:00<00:00, 651.91 examples/s]


In [10]:
train_dataset=dataset['train']
val_dataset=dataset['validation']

## 准备lora

In [11]:
from peft import LoraConfig, TaskType,PeftModel,get_peft_model

In [12]:
# 配置lora
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

model = get_peft_model(model, config)

In [13]:
model.print_trainable_parameters()  # 打印可训练参数，确认是否正确

trainable params: 1,146,880 || all params: 597,196,800 || trainable%: 0.1920


In [14]:
# 确保可训练参数的 requires_grad 为 True
for name, param in model.named_parameters():
    if "lora" in name:  # 仅对 LoRA 参数设置 requires_grad=True
        param.requires_grad = True
    else:
        param.requires_grad = False  # 冻结其他参数

## 准备swanlab

In [14]:
import swanlab

In [15]:
swanlab.config.update({
    "model": "Qwen3-0.6B",
    "prompt": PROMPT,
    "data_max_length": MAX_LENGTH,
    })

## 准备accelerate

In [17]:
from accelerate import Accelerator

In [18]:
accelerator=Accelerator()

In [20]:
model,train_dataset,val_dataset=accelerator.prepare(model,train_dataset,val_dataset)

## 准备训练

In [16]:
from transformers import TrainingArguments,Trainer,DataCollatorForSeq2Seq,DataCollatorWithPadding
import torch

In [17]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

In [18]:
args = TrainingArguments(
    output_dir="../output/Qwen3-0.6B",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=10,
    num_train_epochs=2,
    save_steps=400,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=False,
    report_to="swanlab",
    run_name="qwen3-0.6B",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## 开始训练

In [19]:
trainer.train()

swanlab: Tracking run with swanlab version 0.6.4                                   
swanlab: Run data will be saved locally in /home/adminad/zhangdw/workspace/finetuning/transformers/swanlog/run-20250629_042912-a3b1799d
swanlab: 👋 Hi zhangdw156, welcome to swanlab!
swanlab: Syncing run qwen3-0.6B to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@zhangdw156/transformers
swanlab: 🚀 View run at https://swanlab.cn/@zhangdw156/transformers/runs/dga4anhfwnemdfwu7qjkq


Step,Training Loss,Validation Loss
100,0.200000,1.597160


/home/fine/uv/transformers/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=136, training_loss=0.20173054319970748, metrics={'train_runtime': 182.2887, 'train_samples_per_second': 23.765, 'train_steps_per_second': 0.746, 'total_flos': 1.2713575059554304e+16, 'train_loss': 0.20173054319970748, 'epoch': 2.0})

swanlab: \ Updating experiment status...

## 验证结果

In [28]:
len(dataset['validation'])

241

In [29]:
test_text_list = []

In [30]:
messages=[
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{dataset['validation']['question'][0]}"}
]

In [31]:
messages

[{'role': 'system', 'content': '你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。'},
 {'role': 'user',
  'content': '1895年德国物理学教授伦琴的发现对医学影像学的发展有何具体影响？请从技术进步、学科建立和临床应用三个方面进行分析。'}]

In [32]:
text=tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

In [33]:
text

'<|im_start|>system\n你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。<|im_end|>\n<|im_start|>user\n1895年德国物理学教授伦琴的发现对医学影像学的发展有何具体影响？请从技术进步、学科建立和临床应用三个方面进行分析。<|im_end|>\n<|im_start|>assistant\n'

In [34]:
model_inputs = tokenizer([text], return_tensors="pt").to('cuda:0')

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=MAX_LENGTH,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/home/fine/uv/transformers/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [35]:
response

'<think>嗯，用户问的是1895年伦琴发现X射线对医学影像学的影响，特别是从技术、学科建立和临床应用三个方面的具体影响。首先，我需要回忆一下伦琴的贡献以及当时医学影像学的背景。\n\n首先，技术进步方面，伦琴的X射线发现了医学影像学的基础，这可能涉及他如何解释X射线的穿透和成像机制。记得他在1895年发表了一篇论文，提到X射线可以产生影像，这可能就是早期医学影像的雏形。所以技术进步方面，伦琴的工作直接推动了X射线成像技术的发展，比如X射线成像仪的发明，这可能让医生能够用X射线来诊断疾病，比如肺结核、骨折等，这应该属于技术进步的一部分。\n\n接下来是学科建立，伦琴的发现可能促使了医学影像学的学科化，比如形成了影像学作为独立的学科。之前的医学可能更关注解剖学和病理学，而影像学则需要专门的理论和实验方法。所以学科建立方面，可能需要提到影像学作为独立学科的成立，以及相关领域的研究，比如放射学的发展。\n\n临床应用方面，伦琴的工作对诊断疾病有帮助，比如肺结核、癌症、骨折等。他可能还发现了X射线成像仪的原理，这使得医生能够更准确地诊断疾病，从而改善医疗诊断的准确性和效率。另外，可能还涉及到放射治疗，但用户的问题主要集中在临床应用，所以需要确认是否直接提到放射治疗。\n\n不过，用户的问题可能希望更具体，比如伦琴如何具体应用X射线成像到临床，或者他可能与其他科学家合作，比如在1901年发表论文中提到的伦琴和阿诺德的研究，但用户的问题可能不需要涉及这些细节，只要从技术、学科和临床三个角度回答。\n\n可能需要检查伦琴的贡献是否确实属于医学影像学，比如他是否在1895年发表的论文中详细描述了X射线成像的原理，这可能就是技术进步中的关键点。另外，学科建立方面，影像学作为独立学科的成立，可能需要提到放射学作为学科的兴起，以及影像学与其他学科如医学、物理学、工程学的结合。\n\n临床应用方面，可能提到X射线成像仪的发明，让医生能够通过X射线观察病灶，从而更早诊断疾病，比如肺结核。此外，可能还涉及放射治疗的应用，但用户的问题可能更关注诊断，所以需要确认是否准确。\n\n总结下来，技术进步方面，伦琴的工作直接推动了X射线成像技术的发明，这使得医学影像学成为可能。学科建立方面，影像学作为独立学科的出现，以及放射学的发展。临床应用方面，X射线成像的应用，帮助诊断疾病，提高诊断准确性。\n

In [36]:
response_text = f"""
Question: {{dataset['validation']['question'][0]}}
LLM:{response}
"""

In [37]:
response_text

"\nQuestion: {dataset['validation']['question'][0]}\nLLM:<think>嗯，用户问的是1895年伦琴发现X射线对医学影像学的影响，特别是从技术、学科建立和临床应用三个方面的具体影响。首先，我需要回忆一下伦琴的贡献以及当时医学影像学的背景。\n\n首先，技术进步方面，伦琴的X射线发现了医学影像学的基础，这可能涉及他如何解释X射线的穿透和成像机制。记得他在1895年发表了一篇论文，提到X射线可以产生影像，这可能就是早期医学影像的雏形。所以技术进步方面，伦琴的工作直接推动了X射线成像技术的发展，比如X射线成像仪的发明，这可能让医生能够用X射线来诊断疾病，比如肺结核、骨折等，这应该属于技术进步的一部分。\n\n接下来是学科建立，伦琴的发现可能促使了医学影像学的学科化，比如形成了影像学作为独立的学科。之前的医学可能更关注解剖学和病理学，而影像学则需要专门的理论和实验方法。所以学科建立方面，可能需要提到影像学作为独立学科的成立，以及相关领域的研究，比如放射学的发展。\n\n临床应用方面，伦琴的工作对诊断疾病有帮助，比如肺结核、癌症、骨折等。他可能还发现了X射线成像仪的原理，这使得医生能够更准确地诊断疾病，从而改善医疗诊断的准确性和效率。另外，可能还涉及到放射治疗，但用户的问题主要集中在临床应用，所以需要确认是否直接提到放射治疗。\n\n不过，用户的问题可能希望更具体，比如伦琴如何具体应用X射线成像到临床，或者他可能与其他科学家合作，比如在1901年发表论文中提到的伦琴和阿诺德的研究，但用户的问题可能不需要涉及这些细节，只要从技术、学科和临床三个角度回答。\n\n可能需要检查伦琴的贡献是否确实属于医学影像学，比如他是否在1895年发表的论文中详细描述了X射线成像的原理，这可能就是技术进步中的关键点。另外，学科建立方面，影像学作为独立学科的成立，可能需要提到放射学作为学科的兴起，以及影像学与其他学科如医学、物理学、工程学的结合。\n\n临床应用方面，可能提到X射线成像仪的发明，让医生能够通过X射线观察病灶，从而更早诊断疾病，比如肺结核。此外，可能还涉及放射治疗的应用，但用户的问题可能更关注诊断，所以需要确认是否准确。\n\n总结下来，技术进步方面，伦琴的工作直接推动了X射线成像技术的发明，这使得医学影像学成为可能。学科建立方面

In [38]:
import random
test_text_list = []
for i in random.sample(range(len(dataset['validation'])),3):
    messages=[
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{dataset['validation']['question'][0]}"}
    ]
    text=tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda:0')

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=MAX_LENGTH,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    response_text = f"""Question: {dataset['validation']['question'][i]}\nLLM:{response}"""

    test_text_list.append(swanlab.Text(response_text))
    

In [39]:
swanlab.log({"Prediction": test_text_list})

{'Prediction': ['Question: 医生，我最近在深入研究神经递质的失活过程，特别是去甲肾上腺素（NA）的失活机制。能否详细解释一下NA失活的主要机制，包括摄取1（neuronal uptake）的具体过程，以及NA被摄取回神经末梢后的命运，特别是它如何被单胺氧化酶（MAO）破坏？\nLLM:<think>嗯，用户问的是1895年伦琴对医学影像学的影响，特别是从技术进步、学科建立和临床应用三个方面。首先，我需要回忆一下伦琴的贡献。记得他发明了X射线，这应该是一个关键点。然后要分三个方面来分析。\n\n首先，技术进步方面。伦琴的X射线技术应该改变了医疗诊断的方式，比如X光成像。之前人们用的是玻璃管，现在X射线可以直接穿透物体，这样能更清晰地显示内部结构。可能还要提到成像的准确性，比如可以区分不同组织，帮助诊断骨折、肿瘤等。另外，X射线的高能量和穿透性可能让医生能够观察到更深层的结构，比如肺部或骨骼，这可能对诊断肺结核或骨髓炎有帮助。这部分需要详细点，说明具体技术如何应用，比如在放射科的应用。\n\n接下来是学科建立。伦琴的发明催生了放射医学，也就是放射物理学和放射医学的交叉学科。这可能包括放射治疗、医学影像学的进一步发展，比如核医学和放射化学。需要提到放射医学的起源，比如放射学作为一门独立的科学，可能在1895年之后逐渐形成。另外，可能还涉及放射治疗的发展，比如放射治疗的使用，以及如何将X射线用于治疗疾病，比如癌症。这部分要说明学科如何发展，可能包括放射治疗的应用，比如在放射治疗中的作用。\n\n第三点是临床应用。伦琴的X射线技术对诊断和治疗有重要影响。比如在诊断疾病时，X射线可以显示骨骼的结构变化，帮助诊断骨折、肿瘤等。同时，X射线的高密度和高灵敏度可能帮助医生快速发现病灶。另外，可能还有治疗应用，比如用于癌症的放射治疗，但需要确认时间线是否正确。比如，X射线治疗可能在20世纪初才开始临床应用，而伦琴在1895年已经发明了X射线，所以可能在治疗方面需要时间，但技术上已经成熟。另外，可能还有其他应用，比如X射线在手术中的使用，比如透视帮助医生定位器官，但这可能属于放射治疗的一部分。\n\n然后要检查是否覆盖了三个方面的所有点。技术进步方面，X射线的发明和应用，成像技术，比如X光成像。学科建立方面，放射医学的形成，放射治疗的发展。临床应用方面，诊断和治

In [40]:
swanlab.finish()

swanlab: 🏠 View project at https://swanlab.cn/@zhangdw156/transformers
swanlab: 🚀 View run at https://swanlab.cn/@zhangdw156/transformers/runs/skg1htqqz2dbi1gymkypo
swanlab: Waiting for uploading complete
                                                                                                    